# Refernece using state
## input h5ad 

In [6]:
import h5py
import os
os.environ['MPLBACKEND'] = 'Agg'

In [7]:
# Check the structure of our sample H5 files
with h5py.File('virtual_cell/predicted_only_10.h5ad', 'r') as f:
    print(type(f))
    print("Keys in root:", list(f.keys()))
    if 'obsm' in f:
        print("Keys in obsm:", list(f['obsm'].keys()))
        print("Keys in X:", list(f['X'].keys()))
    else:
        print("No 'obsm' group found")

<class 'h5py._hl.files.File'>
Keys in root: ['X', 'layers', 'obs', 'obsm', 'obsp', 'uns', 'var', 'varm', 'varp']
Keys in obsm: []


AttributeError: 'Dataset' object has no attribute 'keys'

In [3]:
! ls virtural_cell

predicted_only_10.h5ad


In [8]:

# Check the structure of one of COLAB H5 files
with h5py.File('/workspace/training_dataset/competition_support_set/competition_train.h5', 'r') as f:
    print(type(f))
    print("Keys in root:", list(f.keys()))
    if 'obsm' in f:
        print("Keys in obsm:", list(f['obsm'].keys()))
        print("Keys in X:", list(f['X'].keys()))
    else:
        print("No 'obsm' group found")

<class 'h5py._hl.files.File'>
Keys in root: ['X', 'layers', 'obs', 'obsm', 'obsp', 'uns', 'var', 'varm', 'varp']
Keys in obsm: []
Keys in X: ['data', 'indices', 'indptr']


## Model download
### Download trained parameteres (1.8G) and other necessary files

In [9]:
# download method 1  
! wget https://huggingface.co/arcinstitute/ST-Parse/blob/main/final.ckpt 

/usr/bin/sh: 1: wget: not found


In [11]:
! pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [huggingface_hub] [huggingface_hub]


In [13]:
# download method 2 
from huggingface_hub import hf_hub_download

repo_id = "arcinstitute/ST-Parse"
filename = "final.ckpt"

# Download the file, it will be cached and its local path returned
local_path = hf_hub_download(repo_id=repo_id, filename=filename)

print(f"File downloaded to: {local_path}")

final.ckpt:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

File downloaded to: /root/.cache/huggingface/hub/models--arcinstitute--ST-Parse/snapshots/089af1c532c91632eb57a395b1e20b856990f621/final.ckpt


In [14]:
! cp /root/.cache/huggingface/hub/models--arcinstitute--ST-Parse/snapshots/089af1c532c91632eb57a395b1e20b856990f621/final.ckpt virtual_cell/

In [37]:
! state tx infer \
  --output "virtual_cell/prediction_250617/prediction_only_10.h5ad" \
  --model_dir virtual_cell/ST_Parse \
  --checkpoint virtual_cell/ST_Parse/final.ckpt \
  --adata "virtual_cell/predicted_only_10.h5ad" \
  --pert_col "target_gene"

INFO:state._cli._tx._infer:Loaded config from virtual_cell/ST_Parse/config.yaml
INFO:state._cli._tx._infer:Loading model from checkpoint: virtual_cell/ST_Parse/final.ckpt
PertSetsPerturbationModel(
  (loss_fn): SamplesLoss()
  (gene_decoder): LatentToGeneDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=2000, out_features=1024, bias=True)
      (1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=1024, out_features=1024, bias=True)
      (5): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (6): GELU(approximate='none')
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=1024, out_features=512, bias=True)
      (9): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (10): GELU(approximate='none')
      (11): Dropout(p=0.1, inplace=False)
      (12): Linear(in_features=512, out_features=2000, bias=True)
      (13): ReLU

In [33]:
repo_id = "arcinstitute/ST-Parse"
filename = "var_dims.pkl"

# Download the file, it will be cached and its local path returned
local_path = hf_hub_download(repo_id=repo_id, filename=filename)

print(f"File downloaded to: {local_path}")

var_dims.pkl:   0%|          | 0.00/25.7k [00:00<?, ?B/s]

File downloaded to: /root/.cache/huggingface/hub/models--arcinstitute--ST-Parse/snapshots/089af1c532c91632eb57a395b1e20b856990f621/var_dims.pkl


In [30]:
! cp /root/.cache/huggingface/hub/models--arcinstitute--ST-Parse/snapshots/089af1c532c91632eb57a395b1e20b856990f621/cell_type_onehot_map.pkl virtual_cell/ST_Parse 

In [34]:
! cp /root/.cache/huggingface/hub/models--arcinstitute--ST-Parse/snapshots/089af1c532c91632eb57a395b1e20b856990f621/var_dims.pkl virtual_cell/ST_Parse/

In [24]:
! cp competition/first_run/config.yaml virtual_cell/ST_Parse

In [35]:
! ls virtual_cell/ST_Parse/

cell_type_onehot_map.pkl  config.yaml  final.ckpt  var_dims.pkl


In [38]:
import os
from huggingface_hub import snapshot_download

# Define the repository and local directory
repo_id = "arcinstitute/ST-Parse"
local_dir = "virtual_cell/ST_Parse/"

# Download all files from the repository
print(f"Downloading all files from {repo_id}...")
local_path = snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
    local_dir_use_symlinks=False  # This ensures actual files are downloaded, not symlinks
)

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

batch_onehot_map.pkl:   0%|          | 0.00/29.7k [00:00<?, ?B/s]

LICENSE.md: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

.gitattributes: 0.00B [00:00, ?B/s]

data_module.torch:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

MODEL_LICENSE.md: 0.00B [00:00, ?B/s]

pert_onehot_map.pt:   0%|          | 0.00/50.0k [00:00<?, ?B/s]

wandb_path.txt:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

## Inferecne our sample

In [39]:
! state tx infer \
  --output "virtual_cell/prediction_250617/prediction_only_10.h5ad" \
  --model_dir virtual_cell/ST_Parse \
  --checkpoint virtual_cell/ST_Parse/final.ckpt \
  --adata "virtual_cell/predicted_only_10.h5ad" \
  --pert_col "target_gene"

INFO:state._cli._tx._infer:Loaded config from virtual_cell/ST_Parse/config.yaml
INFO:state._cli._tx._infer:Loading model from checkpoint: virtual_cell/ST_Parse/final.ckpt
PertSetsPerturbationModel(
  (loss_fn): SamplesLoss()
  (gene_decoder): LatentToGeneDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=2000, out_features=1024, bias=True)
      (1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=1024, out_features=1024, bias=True)
      (5): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (6): GELU(approximate='none')
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=1024, out_features=512, bias=True)
      (9): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (10): GELU(approximate='none')
      (11): Dropout(p=0.1, inplace=False)
      (12): Linear(in_features=512, out_features=2000, bias=True)
      (13): ReLU

In [42]:
# embedding first, but not solving the error.
! state emb transform \
  --model-folder virtual_cell/ST_Parse \
  --input virtual_cell/predicted_only_10.h5ad \
  --output virtual_cell/predicted_only_10_emb.h5ad

INFO:state._cli._emb._transform:Using model checkpoint: virtual_cell/ST_Parse/final.ckpt
INFO:state._cli._emb._transform:Creating inference object
Traceback (most recent call last):
  File "/root/.local/bin/state", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/root/.local/share/uv/tools/arc-state/lib/python3.11/site-packages/state/__main__.py", line 56, in main
    run_emb_transform(args)
  File "/root/.local/share/uv/tools/arc-state/lib/python3.11/site-packages/state/_cli/_emb/_transform.py", line 39, in run_emb_transform
    protein_embeds = torch.load(embedding_file, weights_only=False, map_location="cpu")
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/.local/share/uv/tools/arc-state/lib/python3.11/site-packages/torch/serialization.py", line 1479, in load
    with _open_file_like(f, "rb") as opened_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/.local/share/uv/tools/arc-state/lib/python3.11/site-p

In [40]:
# using COLAB input for testing
! state tx infer \
  --output "virtual_cell/prediction_250617/prediction_only_10.h5ad" \
  --model_dir virtual_cell/ST_Parse \
  --checkpoint virtual_cell/ST_Parse/final.ckpt \
  --adata "training_dataset/competition_support_set/competition_val_template.h5ad" \
  --pert_col "target_gene"

INFO:state._cli._tx._infer:Loaded config from virtual_cell/ST_Parse/config.yaml
INFO:state._cli._tx._infer:Loading model from checkpoint: virtual_cell/ST_Parse/final.ckpt
PertSetsPerturbationModel(
  (loss_fn): SamplesLoss()
  (gene_decoder): LatentToGeneDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=2000, out_features=1024, bias=True)
      (1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=1024, out_features=1024, bias=True)
      (5): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (6): GELU(approximate='none')
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=1024, out_features=512, bias=True)
      (9): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (10): GELU(approximate='none')
      (11): Dropout(p=0.1, inplace=False)
      (12): Linear(in_features=512, out_features=2000, bias=True)
      (13): ReLU